In [2]:
using LinearAlgebra
using TensorOperations
using Plots

### Canonical Forms



In [3]:
# parameters 
N = 10
d = 3
D = 20

# random MPS
#=
    Order of legs: left-bottom-right
    Note: this is the conventional order used for MPSs in the code.
=#

Mrand = []
push!(Mrand, rand(1, d, D))
for l in 2:N-1
    push!(Mrand, rand(D, d, D))
end
push!(Mrand, rand(D, d, 1))

println(size(Mrand, 1))
println(size(Mrand[2], 1))
println(size(Mrand[2], 2))
println(size(Mrand[2], 3))

10
20
3
20


In [4]:
function left_canonical(M)
    Mcopy = copy(M)
    N = length(Mcopy)

    for l in 1:N
        Taux = Mcopy[l]
        Taux = reshape(Taux, (size(Taux, 1)*size(Taux, 2), size(Taux, 3)))
        
        F = svd(Taux)
        U, S, Vdag = F
        Mcopy[l] = reshape(U, (size(Mcopy[l], 1), size(Mcopy[l], 2), size(U, 2)))
        SVdag = Diagonal(S) * transpose(Vdag)

        if l < N
            @tensor result[i, k, m] := SVdag[i, j] * Mcopy[l+1][j, k, m]
            Mcopy[l+1] = result
        end
    end 
    
    return Mcopy
end

left_canonical (generic function with 1 method)

In [7]:
Mleft = left_canonical(Mrand)
for n in 1:N
    Mdag = conj.(permutedims(Mleft[n], (3, 2, 1)))
    @tensor result[i, l] := Mdag[i, j, k] * Mleft[n][k, j, l]
    MdagM = result
    I = Diagonal(ones(size(Mleft[n], 3)))
    diff = maximum(abs.(MdagM .- I))
    println("l = $n, : max(|M[n] * M[n]^† - I|) = $diff")
end

l = 1, : max(|M[n] * M[n]^† - I|) = 1.4432899320127035e-15
l = 2, : max(|M[n] * M[n]^† - I|) = 1.9984014443252818e-15
l = 3, : max(|M[n] * M[n]^† - I|) = 2.4424906541753444e-15
l = 4, : max(|M[n] * M[n]^† - I|) = 1.6653345369377348e-15
l = 5, : max(|M[n] * M[n]^† - I|) = 1.5543122344752192e-15
l = 6, : max(|M[n] * M[n]^† - I|) = 1.3322676295501878e-15
l = 7, : max(|M[n] * M[n]^† - I|) = 2.886579864025407e-15
l = 8, : max(|M[n] * M[n]^† - I|) = 2.4424906541753444e-15
l = 9, : max(|M[n] * M[n]^† - I|) = 1.9984014443252818e-15
l = 10, : max(|M[n] * M[n]^† - I|) = 4.440892098500626e-16


In [ ]:


println("l=$n, Mdag dimensions: $(size(Mdag))")

UndefVarError: UndefVarError: `n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.